In [1]:
import os
import torch
import mlflow
from gpt import GPTLanguageModel
from gpt_params import tokenizer

/Users/joaquinpolonuer/Documents/venvs/ml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credentials.json"
mlflow.set_tracking_uri(uri="http://34.176.94.221:5000")


In [3]:
device = (
    "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
)

experiment = "Transformer"
run_id = "17825e172dbe48b8934c57b59e85d83c"

gpt_model = GPTLanguageModel.load_from_mlflow(experiment, run_id, device)

/Users/joaquinpolonuer/Documents/exactas/nlp/NLP-monosemanticity/gpt.py:151: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(local_model_path, map_location=

In [4]:
gpt_model.eval()
idx = (
    torch.tensor(tokenizer.encode("The quick brown fox jumps over the lazy dog"), dtype=torch.long)
    .unsqueeze(0)
    .to(device)
)
out = gpt_model.generate(idx, 100)

print(tokenizer.decode(out.squeeze().tolist()))

The quick brown fox jumps over the lazy dog , and a historical plan , but sometimes either completing order to the lower amount is one single of 2 @.@ 7 miles per last . This exists in Muslim species zone in the area where Folwood is considering a new goal for his book and by the Rebellion of the Yiscovered Assembly . The events of the Virgin 's result in help during the tournament . A few F @-@ 480 film was renamed the Military Vouan Association Carah in November 1966 for the first @-@ finals


In [8]:
gpt_model.eval()
idx = (
    torch.tensor(tokenizer.encode("Hello my friend"), dtype=torch.long)
    .unsqueeze(0)
    .to(device)
)
out = gpt_model.greedy_generate(idx, 100)

print(tokenizer.decode(out.squeeze().tolist()))

Hello my friend , and the character 's character , and the song was a " one of the most popular music video , and the song was released on the album . 
 = = = = 
 = = = = = 
 The song was released on the album , and was released on the album , and was released on the album . 
 = = = = = 
 = = = = = 
 The album was released on the album , and was released on the album , and
